本实例演示使用MedLvmWorkflow训练resnet50模型，MedLvmWorkflow默认提供常用的开源医疗数据集

## 准备数据集
首先，从 opendatalab 下载 Camvid 数据集:

In [ ]:
# https://opendatalab.com/CamVid

  pip install openxlab #安装

  pip install -U openxlab #版本升级
  
  openxlab login #进行登录，输入对应的AK/SK
  
  openxlab dataset info --dataset-repo OpenDataLab/CamVid #数据集信息查看

  openxlab dataset ls --dataset-repo OpenDataLab/CamVid #数据集文件列表查看

  openxlab dataset get --dataset-repo OpenDataLab/CamVid #数据集下载
  
  openxlab dataset download --dataset-repo OpenDataLab/CamVid --source-path /README.md --target-path /path/to/local/folder #数据集文件下载

### 创建模型

In [1]:
import sys
sys.path.append('../')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from MedLvmWorkflow.dataset.segmentation.demoDataset import create_palette
from MedLvmWorkflow.models.modelFactory import createTrainModel
color_to_class=create_palette("../data/CamVid/class_dict.csv")
num_class=len(color_to_class.items())
trainModel =createTrainModel("deeplabv3_resnet50_pretrained",num_class)

### 加载数据集

In [2]:

import MedLvmWorkflow
from MedLvmWorkflow.dataset.segmentation.demoDataset import CamVid,create_palette

train_set = CamVid(
    '../data/CamVid',
    img_folder='train',
    mask_folder='train_labels',
    transform=trainModel.transform_img,
    target_transform=trainModel.transform_lable)

valid_set = CamVid(
    '../data/CamVid',
    img_folder='val',
    mask_folder='val_labels',
    transform=trainModel.transform_img,
    target_transform=trainModel.transform_lable)
# print(color_to_class)
# print(num_class)

### 开始训练

In [3]:

from MedLvmWorkflow.trainClass.train import TrainTool
trainProcess=TrainTool()
trainProcess.startTrain(trainModel,
                        train_set,
                        valid_set,
                        batch_size=4,
                        max_epochs=20,
                        checkpoint_interval=10,
                        color_to_class=color_to_class,work_dir='./trainlogs/train_deeplabv3_resnet50')

06/14 13:18:28 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.0 (default, Nov 15 2020, 14:28:56) [GCC 7.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 15160592
    GPU 0: NVIDIA GeForce RTX 2080 Ti
    CUDA_HOME: /usr/local/cuda-11.6
    NVCC: Cuda compilation tools, release 11.6, V11.6.55
    GCC: gcc (Ubuntu 7.5.0-6ubuntu2) 7.5.0
    PyTorch: 2.1.2+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-

KeyboardInterrupt: 

### 推理

In [1]:
import sys
sys.path.append('../')

from MedLvmWorkflow.dataset.segmentation.demoDataset import create_palette
from MedLvmWorkflow.models.modelFactory import createTrainModel,createInferModel
color_to_class=create_palette("../data/CamVid/class_dict.csv")
num_class=len(color_to_class.items())
inferModel =createInferModel("deeplabv3_resnet50_pretrained",num_class,backboneModlePertrained="./epoch_20.pth")

print(inferModel)

TypeError: 'dict' object is not callable

### 训练dinov2

In [7]:

from MedLvmWorkflow.trainClass.train import TrainTool
from MedLvmWorkflow.models.modelFactory import createTrainModel
trainProcess=TrainTool()
trainModel =createTrainModel("dinov2_s_pretrained",num_class,"conv_s")
train_set = CamVid(
    '../data/CamVid',
    img_folder='train',
    mask_folder='train_labels',
    transform=trainModel.transform_img,
    target_transform=trainModel.transform_lable)

valid_set = CamVid(
    '../data/CamVid',
    img_folder='val',
    mask_folder='val_labels',
    transform=trainModel.transform_img,
    target_transform=trainModel.transform_lable)
trainModel.optim["lr"]=0.0007
trainProcess.startTrain(trainModel,
                        train_set,
                        valid_set,
                        batch_size=8,
                        color_to_class=color_to_class,work_dir='./trainlogs/train_dinov2')

Using cache found in /home/botao/.cache/torch/hub/facebookresearch_dinov2_main
/home/botao/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/botao/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/botao/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


06/13 17:31:03 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.0 (default, Nov 15 2020, 14:28:56) [GCC 7.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 34359312
    GPU 0: NVIDIA RTX A4000
    GPU 1: NVIDIA GeForce RTX 2080 Ti
    CUDA_HOME: /usr/local/cuda-11.6
    NVCC: Cuda compilation tools, release 11.6, V11.6.55
    GCC: gcc (Ubuntu 7.5.0-6ubuntu2) 7.5.0
    PyTorch: 2.1.2+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;

/home/botao/anaconda3/envs/LVMWorkFlow/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


06/13 17:31:08 - MedLvmWorkflow - INFO - Epoch(train) [1][10/46]  lr: 7.0000e-04  eta: 0:01:39  time: 0.4504  data_time: 0.2548  memory: 505  loss: 2.8780
06/13 17:31:11 - MedLvmWorkflow - INFO - Epoch(train) [1][20/46]  lr: 7.0000e-04  eta: 0:01:19  time: 0.3774  data_time: 0.2468  memory: 505  loss: 2.7731
06/13 17:31:14 - MedLvmWorkflow - INFO - Epoch(train) [1][30/46]  lr: 7.0000e-04  eta: 0:01:10  time: 0.3520  data_time: 0.2431  memory: 505  loss: 2.7331
06/13 17:31:17 - MedLvmWorkflow - INFO - Epoch(train) [1][40/46]  lr: 7.0000e-04  eta: 0:01:04  time: 0.3392  data_time: 0.2412  memory: 505  loss: 2.7142
06/13 17:31:19 - MedLvmWorkflow - INFO - Exp name: 20240613_173102
06/13 17:31:19 - MedLvmWorkflow - INFO - Saving checkpoint at 1 epochs


/home/botao/anaconda3/envs/LVMWorkFlow/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


...

/home/botao/anaconda3/envs/LVMWorkFlow/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


06/13 17:31:50 - MedLvmWorkflow - INFO - Epoch(val) [2][10/12]    eta: 0:00:01  time: 0.7612  data_time: 0.6559  memory: 571  
06/13 17:31:51 - MedLvmWorkflow - INFO - Epoch(val) [2][12/12]    t/iou: 0.5412  data_time: 0.5910  time: 0.6967
06/13 17:31:54 - MedLvmWorkflow - INFO - Epoch(train) [3][10/46]  lr: 7.0000e-04  eta: 0:00:40  time: 0.3024  data_time: 0.2370  memory: 571  loss: 2.6557
06/13 17:31:57 - MedLvmWorkflow - INFO - Epoch(train) [3][20/46]  lr: 7.0000e-04  eta: 0:00:37  time: 0.3012  data_time: 0.2358  memory: 505  loss: 2.6427
06/13 17:32:00 - MedLvmWorkflow - INFO - Epoch(train) [3][30/46]  lr: 7.0000e-04  eta: 0:00:33  time: 0.3008  data_time: 0.2354  memory: 505  loss: 2.6390
06/13 17:32:03 - MedLvmWorkflow - INFO - Epoch(train) [3][40/46]  lr: 7.0000e-04  eta: 0:00:30  time: 0.3015  data_time: 0.2361  memory: 505  loss: 2.6366
06/13 17:32:05 - MedLvmWorkflow - INFO - Exp name: 20240613_173102
06/13 17:32:05 - MedLvmWorkflow - INFO - Saving checkpoint at 3 epochs


/home/botao/anaconda3/envs/LVMWorkFlow/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


06/13 17:32:13 - MedLvmWorkflow - INFO - Epoch(val) [3][10/12]    eta: 0:00:01  time: 0.7497  data_time: 0.6443  memory: 571  
06/13 17:32:14 - MedLvmWorkflow - INFO - Epoch(val) [3][12/12]    t/iou: 0.5503  data_time: 0.5932  time: 0.6985
06/13 17:32:17 - MedLvmWorkflow - INFO - Epoch(train) [4][10/46]  lr: 7.0000e-04  eta: 0:00:25  time: 0.3013  data_time: 0.2359  memory: 571  loss: 2.6347
06/13 17:32:20 - MedLvmWorkflow - INFO - Epoch(train) [4][20/46]  lr: 7.0000e-04  eta: 0:00:22  time: 0.3009  data_time: 0.2355  memory: 505  loss: 2.6330
06/13 17:32:23 - MedLvmWorkflow - INFO - Epoch(train) [4][30/46]  lr: 7.0000e-04  eta: 0:00:19  time: 0.2999  data_time: 0.2345  memory: 505  loss: 2.6322


KeyboardInterrupt: 